In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
using NonlinearSolve
import DifferentialEquations as DE
include("multiharmonic_balance.jl");
using MethodOfLines
using Latexify

In [ ]:
gamma = 0;
omega = 20.0;
gamma3 = 0.0;
g0::Float64 = 9.80665; # m / s^2
height = 5.0; # m

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
yleft = 0.0;
yright = 1.0;
Nt = 5
Nx = Ny = 100;
harmonics = 4; # number of harmonics
order = 2;
stepx = (xright-xleft)/Nx;
stepy = (yright - yleft)/Ny;
u0 = 0.01 * randn((Nx+1) * (Ny+1) * harmonics * 2);
N = (Nx+1) * (Ny+1);

In [ ]:
# Define symbolics
Model.@parameters x, y, t;

const Dy = Model.Differential(y)
const Dx = Model.Differential(x);
const Dt = Model.Differential(t);

In [ ]:
function build_problem(x, y, t, omega, harmonics, xleft, xright, yleft, yright, gamma, gamma3)
    vars, var_exprs, u = create_ansatz((x, y), t, omega, harmonics);
    bcs = create_bcs(vars, ((xleft, xright), (yleft, yright)), (x, y), 0.0);
    F = 50 * exp(-40*(x^2)) * sin(omega*t);
    pde::Symbolics.Num = Dt(Dt(u)) - 9*(Dx(Dx(u)) + Dy(Dy(u))) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;
    return pde, bcs, var_exprs, vars;
end

In [ ]:
@time begin
pde, bcs, var_exprs, vars = build_problem(x, y, t, omega, harmonics, xleft, xright, yleft, yright, gamma, gamma3);
end;

In [ ]:
@time begin
expanded = expand_trig_jl(pde, t, omega);
end;

In [ ]:
@time begin
eqns = make_residual(expanded, harmonics, omega, t);
end;

In [ ]:
@time begin
sym_eqs = map(transform_sym(Nx, Ny) ∘ Meta.parse ∘ string, eqns)
end;

In [ ]:
@time begin
resid = create_residual_function(Nx, Ny, sym_eqs, vars)
end;

In [ ]:
residual! = eval(resid);
R = similar(u0);
jac_sparsity = Symbolics.jacobian_sparsity(
    (R, U) -> residual!(R, U, [stepx, stepy]), 
    R, u0
);

In [ ]:
# Create problem with sparsity
f = NonlinearFunction(residual!; jac_prototype=float.(jac_sparsity));
prob = NonlinearProblem(f, u0, [stepx, stepy]);

# Solve with sparse Jacobian
sol = solve(prob, NewtonRaphson(), reltol=1e-5, abstol=1e-5);

In [ ]:
solutions = [reshape(sol.u[(k-1)*N+1:k*N], Nx+1, Ny+1) for k in 1:(2*harmonics)];

In [ ]:
using Plots
anim = @animate for t in 1:0.1:10
    if t % 100 == 0
        println(t)
    end

    u_new = solutions[1]*0.0
    j = 1
    for i in 1:(2*harmonics)
        if isodd(i)
            u_new .+= solutions[i] .* sin(j * omega * t)
        else
            u_new .+= solutions[i] .* cos(j * omega * t)
            j += 1
        end
    end
    

    heatmap(u_new, clims=(-0.1, 0.1))
    title!("(Solvetime = 296s) Wave Equation with H = $(harmonics), \$\\gamma\$ = $(gamma), \$\\gamma_3\$ = $(gamma3), \$\\omega\$ = $omega", titlefontsize=10)

end

gif(anim, "HB_WE_2D.gif", fps=5)